In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely.geometry as geom
import numpy as np
import h5py as h5
import taxidata as td
import d_curve as dc
import ksegment as ks
import networkx as nx
import ctypes

Seoul = td.Roadnetwork()
s_elist = gpd.read_file(r'.\taxidata\core\network\nodelink\Seoul_Edgelist.csv')
s_link = gpd.read_file(r".\taxidata\core\network\nodelink\Seoul_Links.shp")
for l, e in s_elist.iterrows():
    geom = s_link['geometry'][int(e['EDGE'])-1]
    Seoul.add_edge(int(e['START_NODE']),int(e['END_NODE']),ID = int(e['EDGE']), length = float(e['LENGTH']), geometry = geom)
    Seoul.nodes[int(e['START_NODE'])]['pos'] = (float(e['XCoord']),float(e['YCoord']))
def angle(G, node1, node2):
    '''aspect of node1 as origin, return angle of node2 in sence of polar coordinates.'''
    p1 = G.nodes[node1]['pos']
    p2 = G.nodes[node2]['pos']
    p2_ = (p2[0]-p1[0],p2[1]-p1[1])
    return np.arctan2(p2_[1],p2_[0])
for edge in Seoul.edges(data = True):
    edge[-1]['angle']=angle(Seoul, edge[0],edge[1])
    
pos = Seoul.pos
seoul_dict = dict(Seoul.pos)
nodeid = np.array(list(seoul_dict.keys()))
nodelist= np.array(list(seoul_dict.values()))
data_raw = td.Dataset('2013-12-12.hdf5')
taxi_raw = data_raw.get_array(fields=['lat','lon'],num = 1,random = False)
taxi = td.toUTM(taxi_raw['lat'],taxi_raw['lon'])
points = np.array(taxi).T
points_f = points.flatten()
points_f=np.array(points_f).astype(np.float32)

grid_200 = [[]for j in range(743*734)]
for nodepos, nid in zip(nodelist, nodeid):
    grid_200_id = nodepos//200 - [1234,20400]
    grid_200[int(grid_200_id[0] + grid_200_id[1]*734)].append(nid)

#node로 segment 만들기
def k(start_node):
    seg = td.k_segments_strict_bfs_with_length(Seoul, start_node, 800)
    len_seg = len(seg)
    list_seg = [[start_node]for l in range(len_seg)]
    n = 0
    for i in seg:
        for j in i.edges():
            list_seg[n].append(j[1])
        n += 1
    return list_seg

#nodeid > node좌표
def k_xy(start_node):
    a = []
    for i in k(start_node):
        node=np.array([seoul_dict[j] for j in i])
        a.append(node)
    return a

    #trajectory grid set에서 start node 뽑기
def start(grid_set):
    node = []
    for i in grid_set:
        node.append(grid_200[i])
    return node

grid1_set = td.trajectory.grid_set(points)
start_set = start(grid1_set)
seg_id=[]
#start node 세그먼트들의 grid number
seg_xy=[]
#start node 세그먼트들의 xy
for z in start_set:
    for j in z:
        seg_id.append(td.k_segments_strict_bfs_with_length(Seoul, j, 800))
        for i in k_xy(j):
            seg_xy.append(i)
seg_id=sum(seg_id,[])
real_xy=[]
real_id=[]
for i in range(len(seg_xy)):
    if np.isin(td.trajectory.trajectory_grid(seg_xy[i]), grid1_set).all():
        real_xy.append(seg_xy[i])
        real_id.append(seg_id[i])

In [27]:
seg_id=[]
#start node 세그먼트들의 grid number
seg_xy=[]
#start node 세그먼트들의 xy
for z in start_set:
    for j in z:
        seg_id.append(td.k_segments_strict_bfs_with_length(Seoul, j, 800))
        for i in k_xy(j):
            seg_xy.append(i)
seg_id=sum(seg_id,[])
real_xy=[]
real_id=[]
for i in range(len(seg_xy)):
    if np.isin(td.trajectory.trajectory_grid(seg_xy[i]), grid1_set).all():
        real_xy.append(seg_xy[i])
        real_id.append(seg_id[i])

In [4]:
import ctypes
_cdll = ctypes.windll.LoadLibrary("./Dll1.dll")

In [5]:
_cdll.d_curve.argtypes = [np.ctypeslib.ndpointer(dtype=np.float32),\
                          np.ctypeslib.ndpointer(dtype=np.int32),\
                          np.ctypeslib.ndpointer(dtype=np.int32),\
                          ctypes.c_int,\
                          np.ctypeslib.ndpointer(dtype=np.float32),\
                          ctypes.c_int,\
                          ctypes.c_int,\
                          np.ctypeslib.ndpointer(dtype=np.float32),\
                          np.ctypeslib.ndpointer(dtype=np.float32),\
                          ctypes.c_int]

In [28]:
seg_len=[]
seg_start=[]
start_init=0
for i in real_xy:
    seg_start.append(start_init)
    seg_len.append(len(i))
    start_init+=len(i)
    
seg_tot=len(real_xy)
real_xy = [item for sublist in real_xy for item in sublist]
real_xy = [item for sublist in real_xy for item in sublist]

traj_len=int(len(points_f)/2)

real_xy=np.array(real_xy).astype(np.float32)
seg_len=np.array(seg_len).astype(np.int32)
seg_start=np.array(seg_start).astype(np.int32)
prefix=np.zeros(seg_tot).astype(np.float32)
d_c=-np.ones(seg_tot).astype(np.float32)

In [29]:
%%time
d_candi1=_cdll.d_curve(real_xy,seg_len,seg_start,seg_tot,points_f,traj_len,0,prefix,d_c,8)

Wall time: 4.28 s


In [32]:
n=0
for i in d_c:
    for j in i:
        if j==-1:
            n+=1
n

205640864

In [60]:
prefix[22469]

0.0

In [63]:
d_c[22469]

-1.0

In [25]:
np.save('haha',d_c,allow_pickle=False)